<a href="https://colab.research.google.com/github/shpotes/traffic-counter/blob/dev/notebooks/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%cd /content/
!rm -rf traffic-counter
!git clone https://github.com/shpotes/traffic-counter -b dev
%cd traffic-counter/notebooks

/content
Cloning into 'traffic-counter'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 972 (delta 34), reused 45 (delta 15), pack-reused 905
Receiving objects: 100% (972/972), 38.22 MiB | 25.04 MiB/s, done.
Resolving deltas: 100% (169/169), done.
/content/traffic-counter/notebooks


In [9]:
!pip install -r ../requirements.txt

In [0]:
import sys
import os
import json
from itertools import groupby
from operator import itemgetter
from typing import List, Tuple, Dict

import gin
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model, clone_model
from tensorflow.keras.layers import Input, Conv2D, Concatenate, MaxPool2D, Activation, Reshape
from tensorflow.keras.applications import VGG16

sys.path.append('..')

from vehicle_nowcasting.data.data_loader import build_source_from_metadata, make_dataset, generate_anchors
from vehicle_nowcasting.data.generate_anchors import generate_anchors as gen_anch
from vehicle_nowcasting.utils import iou, plot_bb, change_box_order, compute_stride_from_receptive_field

%matplotlib inline

In [0]:
main_dir = '..'
data_dir = os.path.join(main_dir, 'data')
metadata = pd.read_csv('../data/metadata_test.csv')
train = metadata[metadata.split == 'train']
val = metadata[metadata.split == 'val']
label_map = json.load(open('../data/label_map.json', 'r'))

train_sources = build_source_from_metadata(train, label_map, data_dir, mode='train')
val_sources = build_source_from_metadata(val, label_map, data_dir, mode='val')

In [25]:
train_ds = make_dataset(train_sources, mode='rpn', num_epochs=200)
val_ds = make_dataset(val_sources, mode='rpn')

15
15


In [0]:
def RPN(inputs, k):
    x = Conv2D(256, kernel_size=(3, 3),  
               activation='relu',
               name='window')(inputs)
    cls = Conv2D(2 * k, kernel_size=(1, 1), name='cls_head')(x)
    _, w, h, _ = cls.shape
    cls = Reshape((w, h, 2, k))(cls)
    cls = Activation('softmax')(cls)
    cls = Reshape((w, h, 2 * k))(cls)
    reg = Conv2D(4 * k, kernel_size=(1, 1), name='reg_head')(x)
    return Concatenate()([cls, reg])

In [0]:
k = 15
inputs = Input(shape=(224, 224, 3))
backbone_model = VGG16(input_tensor=inputs,
                       weights='imagenet',
                       include_top=False)
x = backbone_model.layers[-5].output
model = Model(inputs=inputs, outputs=RPN(x, k))

In [28]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
____________________________________________________________________________________________

In [0]:
def l1_smooth(x):
    return tf.where(tf.math.abs(x) < 1, 0.5 * x * x, tf.math.abs(x) - 0.5)
        
def RPN_loss(k, y_true, y_pred):
    
    p_true = y_true[:, :, :, :(2 * k)]
    t_true = y_true[:, :, :, -(4 * k):]
    p_pred = y_pred[:, :, :, :(2 * k)]
    t_pred = y_pred[:, :, :, -(4 * k):]
    
    cls_loss = tf.keras.losses.binary_crossentropy(p_true, p_pred)
    reg_loss = tf.reduce_sum(l1_smooth(t_true - t_pred)) 
    return tf.math.reduce_sum(cls_loss) / k + tf.math.reduce_sum(p_true * reg_loss) / 200

In [0]:
adam = tf.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=adam,
              loss=lambda y_true, y_pred: RPN_loss(15, y_true, y_pred)) # ?? 

In [53]:
hist = model.fit_generator(train_ds.batch(1))

 46/400 [==>...........................] - ETA: 13:11 - loss: 24.9025

KeyboardInterrupt: ignored